# Визуализация знаний, заключенных в сверточной нейронной сети

## Простая сверточная нейронная сеть для Fashion MNIST

Попробуем обучить простую сверточную сеть на датасете Fashion MNIST.

In [1]:
import keras
from keras.datasets import fashion_mnist # Датасет
from keras.models import Sequential, Model # Базовый класс для создания нейронной сети
from keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, LeakyReLU 
from keras.utils import np_utils # Утилиты для one-hot encoding
from keras.regularizers import l1_l2
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt

print(keras.__version__)

Using TensorFlow backend.
D:\Users\maxim\Anaconda3\envs\maxim\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Users\maxim\Anaconda3\envs\maxim\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Users\maxim\Anaconda3\envs\maxim\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Users\maxim\Anaconda3\envs\maxim\lib\site-pa

KeyboardInterrupt: 

Каждому изображению соответствует единственная метка. Так как названия классов не включены в датасет, сохраним их тут для дальнейшего использования при построении изображений:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

Загрузим и обработаем данные

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Заодно давайте посмотрим, как выглядят изображения

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

In [ ]:
train_images = train_images.reshape(60000, 28, 28, 1)
test_images = test_images.reshape(10000, 28, 28, 1)

Объявим простую нейронную сеть

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), kernel_initializer="he_normal", kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), input_shape=(28, 28, 1)),
    BatchNormalization(),
    LeakyReLU(),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), kernel_initializer="he_normal", kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    BatchNormalization(),
    LeakyReLU(),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(64, kernel_initializer="he_normal", kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
    BatchNormalization(),
    LeakyReLU(),
    Dropout(0.4),
    
    Dense(10, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, batch_size=128, epochs=20)

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nТочность на проверочных данных:', test_acc)

### Визуализация фильтров

Возьмем первый сверточный слой в модели и посмотрим на его фильтры.

In [ ]:
filters, biases = model.layers[0].get_weights()

# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

In [ ]:
filters.shape

In [ ]:
# plot first few filters
n_filters, ix = 15, 1
for i in range(n_filters):
    # get the filter
    f = filters[:, :, :, i]
    # plot each channel separately
    ax = plt.subplot(3, 5, ix)
    ax.set_xticks([])
    ax.set_yticks([])
    # plot filter channel in grayscale
    plt.imshow(f[:, :, 0], cmap="gray")
    ix += 1
# show the figure
plt.show()

### Визуализация карт признаков

Теперь попробуем визуализировать активации 

Загрузим изображение из тестовой выборки

In [ ]:
img = test_images[4]
plt.imshow(img.reshape(28, 28), cmap=plt.cm.binary)

Переопределим модель, чтобы она выводила выходы определенный скрытых слоёв

In [ ]:
ixs = [2, 6]
outputs = [model.layers[i].output for i in ixs]
fmodel = Model(inputs=model.inputs, outputs=outputs)

Получим карты признаков

In [1]:
feature_maps = fmodel.predict(img.reshape(1, 28, 28, 1))
# plot the output from each block
square = 8
for fmap in feature_maps:
    plt.figure(figsize=(10,10))
    ix = 1
    for _ in range(square//2):
        for _ in range(square):
            # specify subplot and turn of axis
            ax = plt.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            plt.grid(False)
            # plot filter channel in grayscale
            plt.imshow(fmap[0, :, :, ix-1], cmap='gray')
            ix += 1
    # show the figure
    plt.show()

NameError: name 'fmodel' is not defined

### Визуализация тепловых карт

В процесс предсказания классов объектов на изображении, иногда наша модель будет ошибаться и предсказывать некорректные классы, например, вероятность правильной метки будет не максимальной. В подобных случаях будет крайне полезно, если бы мы смогли визуализировать области изображений в свёрточной сети на которые она смотрит для определения класса объекта.

Подобная техника визуализации называется Class Activation Map (карта активаций класса). Один из техник применяемая при CAM это наложение тепловой карты на исходное изображение. Тепловая карта классов активации представляет собой 2D сетку, в каждой ячейке которой располагается значение количества баллов, связанных с конкретным выходным классом, вычисленное для каждой позиции исходного изображения и отображающего важность вклада каждого участка в классификацию объекта выходного класса.

Возьмем изображение из тестовой выборки

In [ ]:
img = test_images[18]
plt.imshow(img.reshape(28, 28)*255, cmap=plt.cm.binary)

Узнаем, к какому классу модель относит данное изображение

In [ ]:
x = img.reshape(1, 28, 28, 1)
preds = model.predict(x)
cur_class = np.argmax(preds)
print("Класс: ", class_names[cur_class], "\nИндекс:", cur_class)

Чтобы узнать, какие части изображения были наиболее похожи на данный класс, воспользуемся алгоритмом Grad-CAM.

In [ ]:
# This is the "african elephant" entry in the prediction vector
cur_class_output = model.output[:, cur_class]

# The is the output feature map of the `block5_conv3` layer,
# the last convolutional layer in VGG16
last_conv_layer = model.get_layer('activation_2')

# This is the gradient of the "african elephant" class with regard to
# the output feature map of `block5_conv3`
grads = K.gradients(cur_class_output, last_conv_layer.output)[0]

# This is a vector of shape (64,), where each entry
# is the mean intensity of the gradient over a specific feature map channel
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# This function allows us to access the values of the quantities we just defined:
# `pooled_grads` and the output feature map of `block5_conv3`,
# given a sample image
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

# These are the values of these two quantities, as Numpy arrays,
# given our sample image of two elephants
pooled_grads_value, conv_layer_output_value = iterate([x])

# We multiply each channel in the feature map array
# by "how important this channel is" with regard to the elephant class
for i in range(64):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# The channel-wise mean of the resulting feature map
# is our heatmap of class activation
heatmap = np.mean(conv_layer_output_value, axis=-1)

Для нужд визуализации нормализуем тепловую карту, приведя в ней значение к диапазону от 0 до 1

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()

В заключение используем библиотеку OpenCV, чтобы получить изображение с наложенной тепловой картой

In [ ]:
import cv2

img = img*255
rgbimg = cv2.merge([img, img, img])

# We resize the heatmap to have the same size as the original image
heatmap = cv2.resize(heatmap, (rgbimg.shape[1], rgbimg.shape[0]))

# We convert the heatmap to RGB
heatmap = np.uint8(255 * heatmap)

# We apply the heatmap to the original image
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

In [ ]:
# 0.7 here is a heatmap intensity factor
superimposed_img = heatmap*0.7 + rgbimg

# Save the image to disk
superimposed_img = cv2.resize(superimposed_img, (540, 540))  
cv2.imwrite("result.jpg", superimposed_img)

![title](result.jpg)

Данный прием визуализации помогает ответить на два важных вопроса:
- Почему сеть решила, что на фотографии изображена сумка?
- Где на фотографии находится сумка?

Интересно отметить, что наиболее сильно активированным оказалось расстояние над сумкой: вероятно, именно по наличию большого количества пустого места в этой области изображения сеть и отличает сумку от других предметов одежды. Т.к. модель очень простая, не стоит удивляться подобным странностям. Далее на примере imagenet будет показан случай, когда модель выделяет действительно хорошие признаки.

## VGG16

Попробуем использовать для визуализации уже обученную для изображений нейронную сеть

Импортируем все необходимое

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import Model
from matplotlib import pyplot
import numpy as np
from numpy import expand_dims
from keras import backend as K

Загрузим модель

In [ ]:
vggmodel = VGG16(weights="imagenet")
vggmodel.summary()

Загрузим и обработаем [изображение](https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2019/02/bird.jpg)

In [ ]:
# load the image with the required shape
bird = load_img('bird.jpg', target_size=(224, 224))

pyplot.figure(figsize=(10, 10))
pyplot.axis('off')
pyplot.imshow(bird)

# convert the image to an array
img = img_to_array(bird)
# expand dimensions so that it represents a single 'sample'
img = expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
img = preprocess_input(img)

### Визуализация карт признаков

Переопределим модель, чтобы она выводила выходы определенный скрытых слоёв

In [ ]:
ixs = [2, 5, 9, 13, 17]
outputs = [vggmodel.layers[i].output for i in ixs]
fmodel = Model(inputs=vggmodel.inputs, outputs=outputs)

Получим карты признаков

In [ ]:
feature_maps = fmodel.predict(img)
# plot the output from each block
square = 8
for fmap in feature_maps:
    # plot all 64 maps in an 8x8 squares
    pyplot.figure(figsize=(10,10))
    ix = 1
    for _ in range(square):
        for _ in range(square):
            # specify subplot and turn of axis
            ax = pyplot.subplot(square, square, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            # plot filter channel in grayscale
            pyplot.imshow(fmap[0, :, :, ix-1], cmap='gray')
            ix += 1
    # show the figure
    pyplot.show()

### Визуализация тепловых карт

Мы уже пробовали применить данный метод к простой сети, узнав, что признаки, выделяемые ею, крайне далеки от идеала. Посмотрим, как покажет себя imagenet.

Передадим наше изображение птицы в сеть и декодируем в удобочитаемый вариант.

In [ ]:
preds = vggmodel.predict(img)
cur_class = np.argmax(preds[0])
print('Predicted:', decode_predictions(preds, top=3)[0])

Для получения тепловой карты также воспользуемся Grad-CAM.

In [ ]:
# This is the "african elephant" entry in the prediction vector
african_elephant_output = vggmodel.output[:, cur_class]

# The is the output feature map of the `block5_conv3` layer,
# the last convolutional layer in VGG16
last_conv_layer = vggmodel.get_layer('block5_conv3')

# This is the gradient of the "african elephant" class with regard to
# the output feature map of `block5_conv3`
grads = K.gradients(african_elephant_output, last_conv_layer.output)[0]

# This is a vector of shape (512,), where each entry
# is the mean intensity of the gradient over a specific feature map channel
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# This function allows us to access the values of the quantities we just defined:
# `pooled_grads` and the output feature map of `block5_conv3`,
# given a sample image
iterate = K.function([vggmodel.input], [pooled_grads, last_conv_layer.output[0]])

# These are the values of these two quantities, as Numpy arrays,
# given our sample image of two elephants
pooled_grads_value, conv_layer_output_value = iterate([img])

# We multiply each channel in the feature map array
# by "how important this channel is" with regard to the elephant class
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# The channel-wise mean of the resulting feature map
# is our heatmap of class activation
heatmap = np.mean(conv_layer_output_value, axis=-1)

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
pyplot.matshow(heatmap)
pyplot.show()

In [ ]:
import cv2

# We use cv2 to load the original image
img = cv2.imread("bird.jpg")

# We resize the heatmap to have the same size as the original image
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

# We convert the heatmap to RGB
heatmap = np.uint8(255 * heatmap)

# We apply the heatmap to the original image
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# 0.4 here is a heatmap intensity factor
superimposed_img = heatmap * 0.4 + img

# Save the image to disk
cv2.imwrite('imagenet_result.jpg', superimposed_img)

![title](imagenet_result.jpg)

Imagenet работает намного более разумнее, чем показанная ранее сеть. Вероятно, наиболее активированная часть брюшка помогают распознать именно эту птицу.